<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/OSI-SAF_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_2_OSI_SAF_sea_ice_file_structure.ipynb" target="_blank">NEXT NOTEBOOK >></a>

<font color="#138D75">**EUMETSAT OSI SAF Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn OSI SAF sea ice parameters : Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://osi-saf.eumetsat.int/register" target="_blank">A EUMETSAT OSI SAF account</a>** if you are using or plan to use the EUMETSAT OSI SAF data.

There are no prerequisite notebooks for this module.
</div>
<hr>

# 1.1 Access Sea Ice Parameters - OSI SAF HL FTP
### Data used 

| Product Description | Product Navigator | OSI SAF website link | OSI SAF Identifier |
|:-----------------:|:-----------------:|:-----------------:|:-----------------:|
| Global Sea Ice Concentration (SSMIS) | EO:EUM:DAT:DMSP:OSI-401-B | <a href="https://osi-saf.eumetsat.int/products/osi-401-b" target="_blank">link</a>   | OSI-401-B |
| Global Sea Ice Edge | EO:EUM:DAT:MULT:OSIEDGBN | <a href="https://osi-saf.eumetsat.int/products/osi-402-d" target="_blank">link</a> | OSI-402-d | 
| Global Sea Ice Type | EO:EUM:DAT:MULT:OSITYGBN | <a href="https://osi-saf.eumetsat.int/products/osi-403-d" target="_blank">link</a> | OSI-403-d | 
| Global Sea Ice Emissivity | EO:EUM:DAT:DMSP:SISE50 | <a href="https://osi-saf.eumetsat.int/products/osi-404-a" target="_blank">link</a> | OSI-404-a | 
| Global Low Resolution Sea Ice Drift | EO:EUM:DAT:MULT:GBL-LR-SIDR | <a href="https://osi-saf.eumetsat.int/products/osi-405-c" target="_blank">link</a> | OSI-405-c | 

### Learning outcomes

At the end of this notebook you will know;
* How to download different OSI SAF Sea Ice Products using FTP access

### Outline

Data from OSI SAF is available through multiple sources. Here we will guide you through FTP access to OSI SAF Sea Ice Data.

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Creating our workspace](#section1)
 1. [Setting login information](#section2)
 1. [Downloading via FTP requests](#section3)

<hr>

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Creating the workspace
[Back to top](#TOC_TOP)

</div>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
# library imports 
import os           # a library that allows to access to basic operating system commands like making directories
import json         # a library that helps with JSON format files
import ftplib       # a library that allows to handle ftp requests

Next we will create a download directory to store the products we will download in this notebook.

In [2]:
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Setting Username and Password
[Back to top](#TOC_TOP)

</div>


We will access OSI SAF data from the OSI SAF High Latitude Centre FTP server.

In order to allow us to download data, we need to provide our credentials. We can do this in two ways; either by creating a file called `.eumetsat_osi_saf_sea_ice_credentials` in our home directory (*option 1 - recommended*) or by supplying our credentials directly in this script (*option 2*). 

#### Option 1: creating  `.eumetsat_osi_saf_sea_ice_credentials` in our home directory

For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system.

In this file we need to add the following information exactly as follows;

```
{
"username": "<provided_username>",
"password": "<provided_password>"
}
```

You must replace `<provided_username>` and `<provided_password>` with the information provided to you after you have registered to the OSI SAF website and mentionned your interest in Sea Ice data. 

Once you have created an <a href="https://osi-saf.eumetsat.int/register" target="_blank">EUMETSAT OSI SAF account</a>, the generic password and credentials to access OSI SAF High Latitude server should have been sent to you via email.

Make sure to save the file without any kind of extension.

Once you have done this, you can read in your credentials using the commands in the following cell. These will be used to generate a time-limited token, which will refresh itself when it expires.

Reading credential information

In [3]:
# read credentials
with open(os.path.join(os.path.expanduser("~"),'.eumetsat_osi_saf_sea_ice_credentials')) as json_file:
    credentials = json.load(json_file)

Loading credentials

In [4]:
username = credentials['username'] 
password = credentials['password']

#### Option 2: provide credentials directly

You can provide your credentials directly as follows; by uncommenting the following lines. 

In [5]:
#username = "<provided_username>"
#password = "<provided_password>"

You must replace `<provided_username>` and `<provided_password>` with the information provided to you after you have registered to the OSI SAF website and mentionned your interest in Sea Ice data. 

Once you have created an <a href="https://osi-saf.eumetsat.int/register" target="_blank">EUMETSAT OSI SAF account</a>, the generic password and credentials to access OSI SAF High Latitude server should have been sent to you via email.

*Note: this method is convenient in the short term, but is not really recommended as you have to put your provided username and password in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.*

<div class="alert alert-danger" role="alert">

## <a id='section3'></a>3. Launch an FTP request
[Back to top](#TOC_TOP)

</div>

### Ice concentration :

Let's start by creating a specific folder to hold the Sea Ice Concentration products

In [6]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_concentration")
os.makedirs(download_dir, exist_ok=True)

Sea Ice Concentration products exist for each hemisphere.

Let's look at the Northern hemisphere Arctic Sea Ice Concentration

In [7]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_concentration/nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [8]:
path = '/prod/ice/conc/'
filename = 'ice_conc_nh_polstere-100_multi_202211081200.nc'

In [9]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open('products/sea_ice_concentration/nh/' + filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere Antarctic Sea Ice Concentration

In [10]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_concentration/sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [11]:
path = '/prod/ice/conc/'
filename = 'ice_conc_sh_polstere-100_multi_202211081200.nc'

In [12]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open('products/sea_ice_concentration/sh/' + filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

### Ice Type :

Let's start by creating a specific folder to hold the Sea Ice Type products

In [13]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_type")
os.makedirs(download_dir, exist_ok=True)

Sea Ice Type products exist for each hemisphere.

Let's look at the Northern hemisphere Arctic Sea Ice Type

In [14]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_type/nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [15]:
path = '/prod/ice/type/'
filename = 'ice_type_nh_polstere-100_multi_202211031200.nc'

In [16]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open('products/sea_ice_type/nh/' + filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere Antarctic Sea Ice Type

In [17]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_type/sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [18]:
path = '/prod/ice/type/'
filename = 'ice_type_sh_polstere-100_multi_202211031200.nc'

In [19]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open('products/sea_ice_type/sh/' + filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

### Ice Edge :

Let's start by creating a specific folder to hold the Sea Ice Edge products

In [20]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_edge")
os.makedirs(download_dir, exist_ok=True)

Sea Ice Edge products exist for each hemisphere.

Let's look at the Northern hemisphere Arctic Sea Ice Edge

In [21]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_edge/nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [22]:
path = '/prod/ice/edge/'
filename = 'ice_edge_nh_polstere-100_multi_202211031200.nc'

In [23]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open('products/sea_ice_edge/nh/' + filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere Antarctic Sea Ice Edge

In [24]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_edge/sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [25]:
path = '/prod/ice/edge/'
filename = 'ice_edge_sh_polstere-100_multi_202211031200.nc'

In [26]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open('products/sea_ice_edge/sh/' + filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

### Ice Emissivity :

Let's start by creating a specific folder to hold the Sea Ice Emissivity products

In [27]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_emissivity")
os.makedirs(download_dir, exist_ok=True)

Sea Ice Emmissivity products exist for each hemisphere.

Let's look at the Northern hemisphere Arctic Sea Ice Emmissivity

In [28]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_emissivity/nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [29]:
path = '/prod/ice/emis/'
filename = 'ice_emis_nh_stere-100_ssmis_202211081200.nc'

In [30]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open('products/sea_ice_emissivity/nh/' + filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

Let's look at the Southern hemisphere Antarctic Sea Ice Emissivity

In [31]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_emissivity/sh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [32]:
path = '/prod/ice/emis/'
filename = 'ice_emis_sh_stere-100_ssmis_202211081200.nc'

In [33]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open('products/sea_ice_emissivity/sh/' + filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

### Ice Drift :

Let's start by creating a specific folder to hold the Sea Ice Drift products

In [34]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_drift/")
os.makedirs(download_dir, exist_ok=True)

Sea Ice Drift products exist for each hemisphere.

Let's look at the Northern hemisphere Arctic Sea Ice Drift

In [35]:
download_dir = os.path.join(os.getcwd(), "products/sea_ice_drift/nh")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [36]:
path = '/prod/ice/drift_lr/merged'
filename = 'ice_drift_nh_polstere-625_multi-oi_202211071200-202211091200.nc'

In [37]:
# connect ftp and download file
ftp = ftplib.FTP("osisaf.met.no") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open('products/sea_ice_drift/nh/' + filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_2_OSI_SAF_sea_ice_file_structure.ipynb" target="_blank">NEXT NOTEBOOK >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>